In [60]:
import astropy.io.fits as fits
import numpy as np
import pylab as plt
from PIL import Image
from tensorflow import keras
from keras import utils
from keras.models import Model 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
hdulist = fits.open('train_data_01.fits')

In [55]:
flux = hdulist[0].data
labels = hdulist[1].data['label']
objid = hdulist[1].data['objid']
labels = utils.to_categorical(labels, 3)
labels = labels.astype(np.float32)
flux = flux.reshape(-1,3000,1).astype(np.float32)

train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=0.1)
train_flux, train_labels, test_flux, test_labels = flux[train_idx], labels[train_idx], flux[test_idx], labels[test_idx]


In [49]:
model = Sequential()  
model.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(3000, 1)))  

model.add(Conv1D(32, kernel_size=3, activation='relu'))    
model.add(MaxPooling1D(pool_size=2))  
#model.add(Dropout(0.1))  
model.add(Flatten())  
model.add(Dense(64, activation='relu'))  
#model.add(Dropout(0.1))  
model.add(Dense(3, activation='softmax'))  
  
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_39 (Conv1D)              │ (None, 2998, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_40 (Conv1D)              │ (None, 2996, 32)       │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 1498, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 47936)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │     3,067,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,069,795 (11.71 MB)

 Trainable params: 3,069,795 (11.71 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.compile(
loss='categorical_crossentropy',
optimizer='Adam',
metrics=['accuracy'])
model.fit(train_flux,train_labels, epochs=5, 
validation_data=(test_flux,test_labels))

Epoch 1/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.6783 - loss: 64.1081 - val_accuracy: 0.8580 - val_loss: 2.3213
Epoch 2/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8874 - loss: 2.5862 - val_accuracy: 0.9290 - val_loss: 1.2033
Epoch 3/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8893 - loss: 5.4643 - val_accuracy: 0.9250 - val_loss: 1.8141
Epoch 4/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9439 - loss: 1.1050 - val_accuracy: 0.9340 - val_loss: 0.4807
Epoch 5/5
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9361 - loss: 0.9207 - val_accuracy: 0.9520 - val_loss: 1.4046


In [68]:
predicted_labels = model.predict(test_flux)
predicted_class = np.argmax(predicted_labels, axis=1)
test_class = np.argmax(test_labels, axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [78]:
test_objid = objid[test_idx].astype(np.float32)

In [79]:
catalog = {'objid':test_objid,'predicted_label':test_class}

In [81]:
df = pd.DataFrame(catalog)

In [82]:
df

,objid,predicted_label
0,4895.0,2
1,8761.0,2
2,5714.0,2
3,4115.0,1
4,3855.0,1
...,...,...
995,453.0,0
996,9067.0,2
997,5625.0,2
998,1670.0,0
